In [1]:
import tensorflow as tf
import pandas as pd

In [4]:
df = pd.read_csv("iris.csv")

In [5]:
df.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [6]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [7]:
df.columns = ['sepal_length', 'sepal_width' , 'petal_length', 'petal_width', 'target']

In [10]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [9]:
df['target'] =  df['target'].apply(int)

In [11]:
y = df['target']

In [12]:
X = df.drop('target',axis=1)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
X.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], dtype='object')

In [18]:
#Feature columns for tensorflow estimator
feat_cols =[]
for col in X.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))

In [19]:
feat_cols

[NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [25]:
input_func = tf.estimator.inputs.pandas_input_fn(x= X_train, y=y_train, batch_size=10, num_epochs= 5, shuffle =True)

In [27]:
classifier= tf.estimator.DNNClassifier(hidden_units= [10,20,10], n_classes= 3, feature_columns= feat_cols )

W0915 14:52:44.303267 140683482093376 estimator.py:1811] Using temporary folder as model directory: /tmp/tmptud7b2qa


In [29]:
#training estimator
classifier.train(input_fn = input_func, steps= 50)

W0915 14:52:59.932372 140683482093376 deprecation.py:323] From /home/shubhangi/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0915 14:52:59.989449 140683482093376 deprecation.py:323] From /home/shubhangi/anaconda3/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/inputs/queues/feeding_queue_runner.py:62: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W0915 14:52:59.994040 140683482093376 deprecation.py:323] From /home/shubhangi/anaconda3/lib/python3.7/site-packages/tensorflow_estimator/python/estimator

In [30]:
pred_fn = tf.estimator.inputs.pandas_input_fn( x= X_test, batch_size = len(X_test), shuffle= False)

In [31]:
predictions = list(classifier.predict(input_fn= pred_fn) )

W0915 15:02:26.576575 140683482093376 deprecation.py:323] From /home/shubhangi/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [33]:
final_pred =[]
for pred in predictions:
    final_pred.append(pred['class_ids'][0])

In [34]:
from sklearn.metrics import classification_report, confusion_matrix

In [35]:
print(confusion_matrix(y_test, final_pred))

[[19  0  0]
 [ 0  8  5]
 [ 0  0 13]]


In [36]:
print(classification_report(y_test, final_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.62      0.76        13
           2       0.72      1.00      0.84        13

   micro avg       0.89      0.89      0.89        45
   macro avg       0.91      0.87      0.87        45
weighted avg       0.92      0.89      0.88        45

